#  Introduction to atomman: NeighborList class

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

The NeighborList class allows all neighbor atoms within a specified cutoff distance to be identified, and provides a nice interface for accessing each atom's coordination number and list of neighbor atoms. 

### 1.1. Theory and methodology

The NeighborList class uses the nlist() function in atomman/core/nlist.py to built the list of neighbor atoms.  First, the routine defines an orthogonal super-box that fully encompasses the system.  The super-box is then divided into cubic bins of dimensions of cutoff.  Each bin is given a list of atom ids corresponding to both the real atoms in the system as well as ghost atoms from periodic replicas of the system that fall inside the bin's boundaries.

For all bins containing at least one real atom, the shortest vector distance between the atoms within that bin and nearby bins are calculated using dvect().  The ids of atoms separated by less than the cutoff distance are added to each other's neighbors list, as long as the ids are not the same or already in the list.

**NOTE:** This routine does not differentiate between the different periodic replicas of the atoms.  As such, the system's dimensions should be larger than the cutoff distance used to ensure that atoms are not neighbors of themselves, or periodic replicas of the same atom.

*Updated version 1.2.6:* cmult and code options removed as underlying cython code is better integrated and now considerably faster.

**Library Imports**

In [1]:
# Standard Python libraries
import os
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.5.0
Notebook executed on 2024-12-11


Construct a 10x10x10 bcc supercell system.

In [2]:
box = am.Box(a=2.8665, b=2.8665, c=2.8665)
atoms = am.Atoms(atype=1, pos=[[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])
system = am.System(atoms=atoms, box=box, scale=True)
system = system.supersize(10, 10, 10)
print(system.natoms)

2000


## 2. Identifying neighbors<a id='section2'></a>

The list of neighbors is computed during class initialization or by calling the build() method.

### 2.1. Initialization

The NeighborList can be initialized either by identifying neighbors using a system and a cutoff, or by reading in data from model.

Parameters

- **system** (*atomman.System, optional*) The system to calculate the neighbor list for. Must be given if model is not given.
- **cutoff** (*float, optional*) Radial cutoff distance for identifying neighbors.  Must be given if model is not given.
- **model** (*str or file-like object, optional*) Gives the file path or content to load.  No other parameters are allowed.    
- **initialsize** (*int, optional*) The number of neighbor positions to initially assign to each atom.  Default value is 20.
- **deltasize** (*int, optional*) Specifies the number of extra neighbor positions to allow each atom when the number of neighbors exceeds the underlying array size.  Default value is 10.

In [3]:
print(system.box)

avect =  [28.665,  0.000,  0.000]
bvect =  [ 0.000, 28.665,  0.000]
cvect =  [ 0.000,  0.000, 28.665]
origin = [ 0.000,  0.000,  0.000]


In [4]:
# Generate a new NeighborList object
neighbors = am.NeighborList(system=system, cutoff=2.5)

### 2.2. The coord attribute

The number of neighbors, i.e. coordination, for each atom can be retrieved using the coord attribute.

In [5]:
print('The coordination of atom 4 =', neighbors.coord[4])
print('Average number of neighbors =', neighbors.coord.mean())

The coordination of atom 4 = 8
Average number of neighbors = 8.0


### 2.3. Per-atom neighbor lists

The list of identified neighbors can be obtained using numerical indexing on the NeighborList object.

In [6]:
print('The neighbor atoms of atom 3 are', neighbors[3])

The neighbor atoms of atom 3 are [  2   4  22  24 202 204 222 224]


Since this is a list of atom indices, it can be easily used to select properties of only the neighbor atoms of one atom.

In [7]:
# Show the positions of the neighbor atoms to atom 4
print(system.atoms.pos[neighbors[4]])

[[ 4.29975  1.43325  1.43325]
 [ 7.16625  1.43325  1.43325]
 [ 4.29975 27.23175  1.43325]
 [ 7.16625 27.23175  1.43325]
 [ 4.29975  1.43325 27.23175]
 [ 7.16625  1.43325 27.23175]
 [ 4.29975 27.23175 27.23175]
 [ 7.16625 27.23175 27.23175]]


The neighbor lists can also be iterated over for each atom

In [8]:
# List the neighbor ids for the first 20 neighbors
for i, nlist in enumerate(neighbors):
    print(nlist)
    
    if i == 19:
        print('...')
        break

[   1   19  181  199 1801 1819 1981 1999]
[  0   2  20  22 200 202 220 222]
[   1    3  181  183 1801 1803 1981 1983]
[  2   4  22  24 202 204 222 224]
[   3    5  183  185 1803 1805 1983 1985]
[  4   6  24  26 204 206 224 226]
[   5    7  185  187 1805 1807 1985 1987]
[  6   8  26  28 206 208 226 228]
[   7    9  187  189 1807 1809 1987 1989]
[  8  10  28  30 208 210 228 230]
[   9   11  189  191 1809 1811 1989 1991]
[ 10  12  30  32 210 212 230 232]
[  11   13  191  193 1811 1813 1991 1993]
[ 12  14  32  34 212 214 232 234]
[  13   15  193  195 1813 1815 1993 1995]
[ 14  16  34  36 214 216 234 236]
[  15   17  195  197 1815 1817 1995 1997]
[ 16  18  36  38 216 218 236 238]
[  17   19  197  199 1817 1819 1997 1999]
[  0  18  20  38 200 218 220 238]
...


In [9]:
# List the neighbor dmag values for the first 10 neighbors
for i, nlist in enumerate(neighbors):
    print(system.dmag(i, nlist))
    
    if i == 9:
        print('...')
        break

[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
[2.48246182 2.48246182 2.48246182 2.48246182 2.48246182 2.48246182
 2.48246182 2.48246182]
...


## 3. build()<a id='section3'></a>

The build() method allows an existing NeighborList object to find neighbors for a new system and cutoff.

Parameters

- **system** (*atomman.System*) The system to calculate the neighbor list for. Must be given if model is not given.
- **cutoff** (*float*) Radial cutoff distance for identifying neighbors.  Must be given if model is not given. 
- **initialsize** (*int, optional*) The number of neighbor positions to initially assign to each atom.  Default value is 20.
- **deltasize** (*int, optional*) Specifies the number of extra neighbor positions to allow each atom when the number of neighbors exceeds the underlying array size.  Default value is 10.

In [10]:
# Identify nearest and next-nearest neighbors with a larger cutoff
neighbors.build(system=system, cutoff=2.9)
print('Average number of neighbors =', neighbors.coord.mean())

Average number of neighbors = 14.0


## 4. dump() and load()<a id='section4'></a>

The list of neighbors can also be saved to a file using dump() and accessed later using load().  This is useful for large systems where regenerating the list of neighbors is time consuming.

### 4.1. dump()

Saves the neighbor list to a file.
        
Parameters

- **fname** (*str*) The file name to save the content to.

In [11]:
neighbors.dump('neighborlist.txt')

with open('neighborlist.txt') as f:
    print(f.read())

# Neighbor list:
# The first column gives an atom index.
# The rest of the columns are the indexes of the identified neighbors.
0 1 2 18 19 20 180 181 199 200 1800 1801 1819 1981 1999
1 0 2 3 19 20 21 22 181 200 201 202 220 222 1801
2 0 1 3 4 22 181 182 183 202 1801 1802 1803 1981 1983
3 1 2 4 5 22 23 24 183 202 203 204 222 224 1803
4 2 3 5 6 24 183 184 185 204 1803 1804 1805 1983 1985
5 3 4 6 7 24 25 26 185 204 205 206 224 226 1805
6 4 5 7 8 26 185 186 187 206 1805 1806 1807 1985 1987
7 5 6 8 9 26 27 28 187 206 207 208 226 228 1807
8 6 7 9 10 28 187 188 189 208 1807 1808 1809 1987 1989
9 7 8 10 11 28 29 30 189 208 209 210 228 230 1809
10 8 9 11 12 30 189 190 191 210 1809 1810 1811 1989 1991
11 9 10 12 13 30 31 32 191 210 211 212 230 232 1811
12 10 11 13 14 32 191 192 193 212 1811 1812 1813 1991 1993
13 11 12 14 15 32 33 34 193 212 213 214 232 234 1813
14 12 13 15 16 34 193 194 195 214 1813 1814 1815 1993 1995
15 13 14 16 17 34 35 36 195 214 215 216 234 236 1815
16 14 15 17 18 36 195 1

### 4.2. load()

Loads the dumped content into an existing NeighborList object.

Parameters
 
- **model** (*str or file-like object*) Gives the file path or content to load.

In [12]:
neighbors.load('neighborlist.txt')
print('Average number of neighbors =', neighbors.coord.mean())

Average number of neighbors = 14.0


### 4.3. Load on initializing

The content can also be loaded during initialization using the model parameter.

In [13]:
neighbors = am.NeighborList(model='neighborlist.txt')
print('Average number of neighbors =', neighbors.coord.mean())

Average number of neighbors = 14.0


**File Cleanup**

In [14]:
os.remove('neighborlist.txt')